## Python introspection

In [1]:
import inspect

In [2]:
print(help(inspect))

Help on module inspect:

NAME
    inspect - Get useful information from live Python objects.

MODULE REFERENCE
    https://docs.python.org/3.11/library/inspect.html
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    This module encapsulates the interface provided by the internal special
    attributes (co_*, im_*, tb_*, etc.) in a friendlier fashion.
    It also provides some help for examining source code and class layout.
    
    Here are some of the useful functions provided by this module:
    
        ismodule(), isclass(), ismethod(), isfunction(), isgeneratorfunction(),
            isgenerator(), istraceback(), isframe(), iscode(), isbuiltin(),
            isroutine() - check object types
   

In [3]:
def func():
    pass

def gen():
    yield

async def afunc():
    pass

In [4]:
inspect.isfunction(func)

True

In [5]:
inspect.isfunction(gen)

True

In [6]:
inspect.isfunction(afunc)

True

In [7]:
inspect.isgeneratorfunction(gen)

True

In [8]:
inspect.isgeneratorfunction(func)

False

In [9]:
inspect.iscoroutinefunction(afunc)

True

In [10]:
inspect.iscoroutinefunction(gen)

False

In [11]:
inspect.iscoroutinefunction(func)

False

In [12]:
def log_dec(func):

    def wrapper(*a, **kw):
        print("call func", func, "with a and kw", a, kw)
        res = func(*a, **kw)
        print("res:", res)
        return res

    return wrapper

In [13]:
def power(num, e=2):
    return num ** e

In [14]:
print(power)

<function power at 0x110097d80>


In [15]:
power

<function __main__.power(num, e=2)>

In [16]:
power(3, e=4)

81

In [17]:
@log_dec
def power(num, e=2):
    return num ** e

In [18]:
power(3, e=4)

call func <function power at 0x1100972e0> with a and kw (3,) {'e': 4}
res: 81


81

In [19]:
print(power)

<function log_dec.<locals>.wrapper at 0x110097560>


In [20]:
power

<function __main__.log_dec.<locals>.wrapper(*a, **kw)>

In [21]:
power.__name__

'wrapper'

In [22]:
power.__module__

'__main__'

In [25]:
from functools import wraps


def log_dec(func):

    @wraps(func)
    def wrapper(*a, **kw):
        print("call func", func, "with a and kw", a, kw)
        res = func(*a, **kw)
        print("res:", res)
        return res

    return wrapper


@log_dec
def power(num, e=2):
    return num ** e

In [26]:
power

<function __main__.power(num, e=2)>

In [27]:
power.__name__

'power'

In [28]:
power.__qualname__

'power'

In [29]:
power(3, 4)

call func <function power at 0x110097a60> with a and kw (3, 4) {}
res: 81


81

In [30]:
print(power)

<function power at 0x110096980>


In [31]:
print(power.__wrapped__)

<function power at 0x110097a60>


In [33]:
inspect.getfullargspec(power)

FullArgSpec(args=[], varargs='a', varkw='kw', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})

In [34]:
def func(a: int, b: str, c: float = 0.5, *, d = None) -> list:
    ...


print(inspect.getfullargspec(func))

FullArgSpec(args=['a', 'b', 'c'], varargs=None, varkw=None, defaults=(0.5,), kwonlyargs=['d'], kwonlydefaults={'d': None}, annotations={'return': <class 'list'>, 'a': <class 'int'>, 'b': <class 'str'>, 'c': <class 'float'>})


In [35]:
async def safe_callback(callback, **kwargs):
    spec = inspect.getfullargspec(callback)
    if spec.varkw:
        accepted_kwargs = kwargs
    else:
        accepted_args = spec.args + spec.kwonlyargs
        accepted_kwargs = {
            name: value
            for name, value in kwargs.items()
            if name in accepted_args
        }

    # result = callback(**accepted_kwargs)
    # if inspect.isawaiable(result):
    #     result = await result

    if inspect.iscoroutinefunction(callback):
        result = await callback(**accepted_kwargs)
    else:
        result = callback(**accepted_kwargs)

    return result

In [37]:
def square(a):
    return a * a


def power(n, e):
    return n ** e


print(
    await safe_callback(square, a=7, b=6, c=7)
)
print(
    await safe_callback(power, a=7, b=6, c=7, n=4, e=3)
)


49
64


In [38]:
import asyncio


async def wait():
    await asyncio.sleep(0.5)
    return 42


print(
    await safe_callback(wait, a=7, b=6, c=7, n=4, e=3)
)

42
